In [1]:
%%capture
!pip install tensorflow==2.17.0

In [2]:
!pip show tensorflow keras

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.12/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 
---
Name: keras
Version: 3.9.2
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: /opt/conda/lib/python3.12/site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow


In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import StratifiedKFold, train_test_split
import json

from tensorflow import keras
from tensorflow.keras.applications import ResNet50

2025-05-19 17:52:39.056132: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 17:52:39.069568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-19 17:52:39.085877: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-19 17:52:39.090483: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 17:52:39.103104: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# Smarter work
def ResNet50PlusPlus(input_shape=(227, 227, 3), output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias.item())

    # Load ResNet50 without top classification layer
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    resnet.trainable = True
    
    model = keras.models.Sequential([
        resnet,
        keras.layers.Dropout(0.5),
        keras.layers.Flatten(),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)
    ])

    return model

In [5]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path)
            img = np.array(img) / 255.0  # Normalize
            images.append(img)
            labels.append(label)

    return images, labels

In [6]:
positive_dir = "training_data/positive"
negative_dir = "training_data/negative"

pos_images, pos_labels = load_images_from_folder(positive_dir, 1)
neg_images, neg_labels = load_images_from_folder(negative_dir, 0)

# Combine data
images = np.array(pos_images + neg_images)
labels = np.array(pos_labels + neg_labels)

In [7]:
def get_class_weights_and_bias(y_data):
    all_sum = len(y_data)
    pos_sum = sum(y_data)
    neg_sum = all_sum - pos_sum

    weight_for_0 = (1 / neg_sum) * (all_sum / 2.0)
    weight_for_1 = (1 / pos_sum) * (all_sum / 2.0)
    
    class_weight = {0: weight_for_0, 1: weight_for_1}
    bias = pos_sum / neg_sum

    return class_weight, bias

In [8]:
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(images, labels, test_size=0.2, random_state=0, stratify=labels)

num_folds = 5
epochs_multiple = 3
num_epochs = 300

num_epoch_batches = num_epochs // epochs_multiple
total_num_epochs = num_folds * num_epochs

loss_metrics = []
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)
for split, (train_index, test_index) in enumerate(kf.split(X_train_final, y_train_final)):
    X_train, X_validate = X_train_final[train_index], X_train_final[test_index]
    y_train, y_validate = y_train_final[train_index], y_train_final[test_index]
    
    class_weight, bias = get_class_weights_and_bias(y_train)

    model = ResNet50PlusPlus(input_shape=(227, 227, 3), output_bias=np.log([bias]))
    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name='accuracy')],
    )

    for epoch_batch in range(num_epoch_batches):
        log_epoch = (epoch_batch + split * num_epoch_batches) * epochs_multiple
        if (log_epoch % 30 == 0 and log_epoch > 0):
            print(f"Epoch {log_epoch}/{total_num_epochs}")
            with open(f"models/loss_metrics_{log_epoch}_{total_num_epochs}.json", "w") as f:
                json.dump(loss_metrics, f)

        X_train_inner, X_validate_inner, y_train_inner, y_validate_inner = train_test_split(X_train, y_train, test_size=0.1, random_state=0, stratify=y_train)
        history = model.fit(
            X_train_inner,
            y_train_inner,
            batch_size=32,
            epochs=epochs_multiple,
            validation_data=(X_validate_inner, y_validate_inner),
            class_weight=class_weight,
        )

        # Store train and validation losses
        loss_fn = keras.losses.BinaryCrossentropy()

        y_train_pred = model.predict(X_train)            
        sample_weights_train = np.where(y_train == 1, class_weight[1], class_weight[0])
        loss_train = float(loss_fn(y_train, y_train_pred, sample_weight=sample_weights_train))

        y_validate_pred = model.predict(X_validate)
        sample_weights_validate = np.where(y_validate == 1, class_weight[1], class_weight[0])
        loss_validate = float(loss_fn(y_validate, y_validate_pred, sample_weight=sample_weights_validate))

        loss_metrics.append((loss_train, loss_validate))
        model.save(f"models/ResNet_model_{log_epoch + epochs_multiple}.keras")

with open(f"models/loss_metrics_{total_num_epochs}_{total_num_epochs}.json", "w") as f:
    json.dump(loss_metrics, f)

Epoch 1/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.8790 - loss: 1.5217 - val_accuracy: 0.8433 - val_loss: 636051.9375
Epoch 2/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9665 - loss: 0.1399 - val_accuracy: 0.8433 - val_loss: 25.9929
Epoch 3/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.9782 - loss: 0.0919 - val_accuracy: 0.8433 - val_loss: 3188.7004
42/42 ━━━━━━━━━━━━━━━━━━━━ 19s 429ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 367ms/step
Epoch 1/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.9715 - loss: 0.1785 - val_accuracy: 0.8433 - val_loss: 121769.7500
Epoch 2/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.9745 - loss: 0.0871 - val_accuracy: 0.1567 - val_loss: 45.9764
Epoch 3/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.9798 - loss: 0.0619 - val_accuracy: 0.8433 - val_loss: 2.2898
42/42 ━━━━━━━━━━━━━━━━━━━━ 11s 255ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 289ms/step
Epoch 1/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.9905 -